In [13]:
'''
READ ME:

This code can be used to pull weather data from the last 30 days using the NDAWN API and generate multiple spatial interpolations with the data.
You will need to input the download save location and location of the project gdb.
The code that writes the output csv file, and removed unecessary rows was written by ChatGPT.
'''

import requests
import arcpy

base_url = "https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=159&station=10&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=201&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=192&station=19&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=172&station=139&station=158&station=23&station=157&station=62&station=86&station=24&station=89&station=126&station=167&station=93&station=183&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=153&station=105&station=82&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=193&station=71&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=69&station=194&station=145&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=57&station=149&station=148&station=202&station=110""" # Required input for notebook function
  
variable = 'ddavt' # Required input for notebook function. Can also be ddmxt for daily max or ddmnt for daily min.
begin_date = '2023-11-07' # Required input for notebook function
quick_pick = '30_d'  # Required input for notebook function. Can also be 7_d, 14_d, 60_d, 90_d, or 100_d days.


api_url = f"{base_url}&variable={variable}&year=2023&ttype=daily&quick_pick={quick_pick}&begin_date={begin_date}&end_date={begin_date}" # Construct the final API URL

In [14]:
response = requests.get(api_url) # Run api request
file_path = r'C:\School\GIS 5571\Local\Lab3 Data\Data\NDAWN_78.csv' # this is a required input field
with open(file_path, 'wb') as file: # write the file to the selected path
    file.write(response.content)

In [15]:
import csv

input_csv_file = r'C:\School\GIS 5571\Local\Lab3 Data\Data\NDAWN_78.csv' # Input file name
output_csv_file = r'C:\School\GIS 5571\Local\Lab3 Data\Data\NDAWN_78_Clean.csv' # Output file name
rows_to_delete = [0, 1, 2, 4]  # Rows to be deleted, note that if the output csv contains multiple tables, more rows will need to be added

with open(input_csv_file, 'r', newline='') as input_file, open(output_csv_file, 'w', newline='') as output_file:
    csv_reader = csv.reader(input_file) # Setting input file as the reader file
    csv_writer = csv.writer(output_file) # Setting output file as the writer file

    for row_index, row in enumerate(csv_reader): # For every row in the reader
        if row_index not in rows_to_delete: # If row is not present in delete list
            csv_writer.writerow(row) # Write row information to new file

In [16]:
output_feature_class = r'C:\School\GIS 5571\Local\Lab3 Data\Lab3 Project\Lab3 Project.gdb\NDAWN_Points' # Points Feature Class
x_field = 'Longitude'  # Long field from csv
y_field = 'Latitude'   # Lat field from csv
arcpy.management.XYTableToPoint(output_csv_file, output_feature_class, x_field, y_field) # Running the tool

<Result 'C:\\School\\GIS 5571\\Local\\Lab3 Data\\Lab3 Project\\Lab3 Project.gdb\\NDAWN_Points'>

In [23]:
averages_class = r"C:\School\GIS 5571\Local\Lab3 Data\Lab3 Project\Lab3 Project.gdb\NDAWN_Averages"
fields_stats = "Latitude FIRST;Longitude FIRST;Avg_Temp MEAN"

arcpy.management.Dissolve(in_features = "NDAWN_Points", out_feature_class = averages_class, dissolve_field = "Station_Name", statistics_fields = fields_stats) # Run this to average the 30 day data to one value per station.

<Result 'C:\\School\\GIS 5571\\Local\\Lab3 Data\\Lab3 Project\\Lab3 Project.gdb\\NDAWN_Averages'>

In [24]:
out_raster_IDW = r"C:\School\GIS 5571\Local\Lab3 Data\Lab3 Project\Lab3 Project.gdb\IDW_Interpolation"
arcpy.ga.IDW(in_features = "NDAWN_Averages", z_field = "MEAN_Avg_Temp", out_raster = out_raster_IDW) # Create the IDW Interpolation

out_raster_Kriging = r"C:\School\GIS 5571\Local\Lab3 Data\Lab3 Project\Lab3 Project.gdb\Kriging_Interpolation"
arcpy.ga.EmpiricalBayesianKriging(in_features="NDAWN_Averages", z_field="MEAN_Avg_Temp", out_raster = out_raster_Kriging)

out_raster_Spline = arcpy.sa.Spline(in_point_features="NDAWN_Averages", z_field="MEAN_Avg_Temp") # Create Spline Interpolation
out_raster_Spline.save(r"C:\School\GIS 5571\Local\Lab3 Data\Lab3 Project\Lab3 Project.gdb\Spline_NDAWN1") # Save Spline Interpolation  